In [7]:
import pandas as pd
import numpy as np
hosp = '/Users/mariama/Documents/GitHub/small-things/urgent_care_project/Hospitals.csv'
# Had to use R to reformat the CSV somehow?? I'm sure there was a better answer, but this worked.
hospital = pd.read_csv(hosp, encoding = "ISO-8859-1")
#del(hospital['Unnamed: 0']) #delete the numbered rows that R added in. 

pop_means = 'pop_means_tract.csv'
center = pd.read_csv(pop_means, encoding = "ISO-8859-1")

In [8]:
er = hospital[hospital.TYPE.isin(['GENERAL ACUTE CARE','CRITICAL ACCESS'])]

In [9]:
# Taken from: https://gist.github.com/rochacbruno/2883505 (Thanks!!)

# Haversine formula example in Python
# Author: Wayne Dyck

import math

def distance(origin, destination):
    lat1, lon1 = origin
    lat2, lon2 = destination
    radius = 6371 # km

    dlat = math.radians(lat2-lat1)
    dlon = math.radians(lon2-lon1)
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = radius * c

    return d

In [10]:
hosp_latlon = pd.DataFrame(columns = ['latlon'])
hll = er[['LAT','LON']].values.tolist()
hosp_latlon['latlon'] = hll

ct_latlon = pd.DataFrame(columns = ['latlon'])
ctll = center[['LATITUDE','LONGITUDE']].values.tolist()
ct_latlon['latlon'] = ctll

In [11]:
# Defining the function to get the closest hospitals
def get_nearest_hosp(ct_locs):
    hosp_locs = hosp_latlon["latlon"]
    dist_list = []
    for item in hosp_locs: 
        dist_list.append(distance(item, ct_locs))
    temp = pd.DataFrame(dist_list)
    #closest_hospital = er.iloc[temp.idxmin()]
    #closest_hospital = temp.min()
    closest_hospital_index = temp.idxmin()
    return closest_hospital_index

# Has to be two functions because .apply will only return one thing.
def get_nearest_hosp_dist(ct_locs):
    hosp_locs = hosp_latlon["latlon"]
    dist_list = []
    for item in hosp_locs: 
        dist_list.append(distance(item, ct_locs))
    temp = pd.DataFrame(dist_list)
    closest_hospital_dist = temp.min()
    return closest_hospital_dist

In [12]:
#How long will this take???
import time
start_time = time.time()

ct_ll  = ct_latlon.head()
ct_ll['latlon'].apply(get_nearest_hosp)
ct_ll['latlon'].apply(get_nearest_hosp_dist)

print((((len(ct_latlon)/5)*(time.time() - start_time))/60), "minutes")


29.743918975194294 minutes


In [13]:
#Applies nearest hospital functions to add two new columns to center.
center['hosp_index'] = ct_latlon['latlon'].apply(get_nearest_hosp)
center['dist'] = ct_latlon['latlon'].apply(get_nearest_hosp_dist)


In [ ]:
#Need to reindex ER, since earlier, I dropped out all non critical access hospitals.
er = er.reset_index(drop=True)

# Merge population center & hospital lists together on the hospital index.
fullset = pd.merge(center, er, left_on = "hosp_index", right_index=True, how = 'left')
fullset.head()
#This is what it would look like without any hospitals having closed - probably a good approximation of 2011.

In [ ]:
#open_hosp = fullset[fullset.STATUS.isin(['OPEN'])]
#closed_hosp = fullset[fullset.STATUS.isin(['CLOSED'])]

In [ ]:
# STAT2 is the status of hospitals that I checked, from a list of hospitals that had closed.
# Combining it with the STATUS that came in the original dataset, 
# to create a test column that aggregates the two.

er['test'] = np.where(((er['STATUS']=="CLOSED")|(er["STAT2"]=='CLOSED')), "CLOSED", "OPEN")

In [ ]:
# Getting the closest hospitals after removing closed hospitals

hosp_latlon = pd.DataFrame(columns = ['latlon'])

# looking for only open hospitals:
er_open = er[er.test.isin(['OPEN'])]
hll = er_open[['LAT','LON']].values.tolist()
hosp_latlon['latlon'] = hll

In [ ]:
ct_current = center
ct_current['hosp_index'] = ct_latlon['latlon'].apply(get_nearest_hosp)
ct_current['dist'] = ct_latlon['latlon'].apply(get_nearest_hosp_dist)

In [ ]:
ct_current.iloc[ct_current['dist'].idxmax()]

In [ ]:
#Need to reindex ER, since earlier, I dropped out all non critical access hospitals.
er_open = er_open.reset_index(drop=True)

# Merge population center & hospital lists together on the hospital index.
only_open = pd.merge(ct_current, er_open, left_on = "hosp_index", right_index=True, how = 'left')


In [ ]:
only_open.to_csv("distance_2018.csv")

## This section will look at only hospitals that have closed that I have verified are closed

In [15]:
closed_er = er[er.STAT2.isin(["CLOSED"])]
closed_hospital_IDs = closed_er["OBJECTID"].tolist()

In [16]:
# using the first set of closest hospitals, find the hospitals that have closed and the associated census tracts
closed_er_cts = fullset[fullset["OBJECTID"].isin(closed_hospital_IDs)]
closed_er_cts = closed_er_cts[['STATEFP', 'COUNTYFP', 'TRACTCE', 'LATITUDE', 'LONGITUDE', "hosp_index", "dist", "OBJECTID", "NAME", "POPULATION_x", "LAT", "LON", "STATUS", "STAT2"]]

In [17]:
#Generate a new list of longitudes and latitudes for only the communities that have seen hospital closures. 
hosp_latlon = pd.DataFrame(columns = ['latlon'])

# looking for only open hospitals:
er_open = er[er.STAT2.isin(['OPEN'])]
hll = er_open[['LAT','LON']].values.tolist()
hosp_latlon['latlon'] = hll

# looking for closest hospitals to census tracts that had seen closed ERs
ct_latlon = pd.DataFrame(columns = ['latlon'])
ctll = closed_er_cts[['LATITUDE','LONGITUDE']].values.tolist()
ct_latlon['latlon'] = ctll

closed_er_cts = closed_er_cts.reset_index(drop=True)

In [19]:
closed_er_cts['new_hosp_index'] = ct_latlon['latlon'].apply(get_nearest_hosp)
closed_er_cts['new_dist'] = ct_latlon['latlon'].apply(get_nearest_hosp_dist)
closed_er_cts['distchange'] = closed_er_cts['new_dist'] - closed_er_cts['dist']
np.mean(closed_er_cts['distchange']) #average increase in travel distance


13.182081888229844

In [20]:
# merge in hospital distances + names+ etc
er_open = er_open.reset_index(drop=True)
closed_er_cts = pd.merge(closed_er_cts, er_open, left_on = "new_hosp_index", right_index=True, how = 'left')

#clean it up a bit
#closed_er_cts =  closed_er_cts[['STATEFP', 'COUNTYFP', 'TRACTCE', 'LATITUDE', 'LONGITUDE', "hosp_index", "dist", "NAME_x", "POPULATION_x", "LAT_x", "LON_x", "STATUS_x", "STAT2_x",
#                                    "new_dist", 'distchange', 'NAME_y', "LAT_y", "LON_y", "BEDS", "STATUS_y"]]
closed_er_cts.to_csv("closed_er_next_closest_full.csv")

in retrospect, i could have probably defined that earlier function better, but hey whatever, it's done now.